In [1]:
import numpy as np
from scipy.special import gamma, iv, modstruve, hyp2f1, erf
from scipy import integrate, interpolate
from scipy.integrate import quad, dblquad, nquad
from QEdark_constants import *

In [66]:
kmpers_nu = 1e5
def etaFromF(vmin,data): 
    """
    takes as input the infile which gives {v,f(v)}
    outputs function eta(vmin)
    
    n.b. takes vE = 232 km/s and vesc = 544 km/s
    """
    vE = 232*kmpers_nu
    vesc = 544*kmpers_nu
    
    # data = np.loadtxt(infile)
    x = data[:,0]*kmpers_nu
    y = data[:,1]
    func = interpolate.interp1d(x,y,kind='linear',bounds_error = False,fill_value = 0) # interpolate to get f(v)
    
    " calculate normalization constant "
    def inttest(vx):
        if vx <= vesc:
            return vx**2*func(vx)
        else:
            return 0
    def bounds():
        return [0.,vesc]
    K_=4*np.pi*nquad(inttest,[bounds])[0]   
    
    if vmin <= vesc - vE:
        def bounds_cosq():
            return [-1,1]
        def bounds_vX(cosq):
            return [vmin, -cosq*vE+np.sqrt((cosq**2-1)*vE**2+vesc**2)]
        def eta(vx,cosq):
            return (2*np.pi/K_)*vx*func(vx**2+vE**2+2*vx*vE*cosq)
        vx = bounds_vX(0.5)[1]*1e-5
        cosq = 0.5
        print ('came here',vx**2+vE**2+2*vx*vE*cosq)
        return nquad(eta, [bounds_vX,bounds_cosq])[0]
        
    elif vesc - vE < vmin <= vesc + vE:
        def bounds_cosq(vx):
            return [-1, (vesc**2-vE**2-vx**2)/(2*vx*vE)] 
        def bounds_vX():
            return [vmin, vE+vesc]
        def eta(cosq,vx):
            return (2*np.pi/K_)*vx*func(vx**2+vE**2+2*vx*vE*cosq)
        return nquad(eta, [bounds_cosq,bounds_vX])[0]
    else:
        return 0

In [67]:
VMIN = 1e5
DATA = np.array([[0,100,200,300,400,500,600,700],[0,1e-3,2e-3,4e-3,3e-3,2e-3,2e-3,0]]).T
# DATA[:,0]
for VMIN in [10e5,100e5,200e5,300e5,400e5,500e5]:
    print (etaFromF(VMIN,DATA))

came here 538249037417978.3
0.0
came here 538249037417978.3
0.0
came here 538249037417978.3
0.0
came here 538249037417978.3
0.0
0.0
0.0


In [2]:
def etaFromF(vmin,infile): 
    """
    takes as input the infile which gives {v,f(v)}
    outputs function eta(vmin)
    
    n.b. takes vE = 232 km/s and vesc = 544 km/s
    """
    vE = 232*kmpers_nu
    vesc = 544*kmpers_nu
    
    data = np.loadtxt(infile)
    x = data[:,0]
    y = data[:,1]
    func = interpolate.interp1d(x,y,kind='linear') # interpolate to get f(v)
    
    " calculate normalization constant "
    def inttest(vx):
        if vx <= vesc:
            return vx**2*func(vx)
        else:
            return 0
    def bounds():
        return [0.,vesc]
    K_=4*np.pi*nquad(inttest,[bounds])[0]   
    
    if vmin <= vesc - vE:
        def bounds_cosq():
            return [-1,1]
        def bounds_vX(cosq):
            return [vmin, -cosq*vE+np.sqrt((cosq**2-1)*vE**2+vesc**2)]
        def eta(vx,cosq):
            return (2*np.pi/K_)*vx*func(vx**2+vE**2+2*vx*vE*cosq)
        return nquad(eta, [bounds_vX,bounds_cosq])[0]
        
    elif vesc - vE < vmin <= vesc + vE:
        def bounds_cosq(vx):
            return [-1, (vesc**2-vE**2-vx**2)/(2*vx*vE)] 
        def bounds_vX():
            return [vmin, vE+vesc]
        def eta(cosq,vx):
            return (2*np.pi/K_)*vx*func(vx**2+vE**2+2*vx*vE*cosq)
        return nquad(eta, [bounds_cosq,bounds_vX])[0]
    else:
        return 0